* Veure
https://discuss.huggingface.co/t/most-efficient-multi-label-classifier/9296/2

* The Artificial Guy - MULTI-LABEL TEXT CLASSIFICATION USING BERT AND PYTORCH
https://www.youtube.com/watch?v=f-86-HcYYi8

* Saurabh Anand - BERT for Multi-Label Classification
https://www.youtube.com/watch?v=JjcxZPNZbUY

* KGP Talkie - 5 - Multi-Label Text Classification Model with DistilBERT and Hugging Face Transformers in PyTorch
https://www.youtube.com/watch?v=ZYc9za75Chk

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
baseDir = '/content/drive/My Drive/TFM-MUECIM'
sys.path.append(baseDir)

In [3]:
import os
import torch
from tfm_LabelLoader import LabelLoader
from tfm_EURLEX57KDataset import EURLEX57KDataset
from torch.utils.data import DataLoader, random_split
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import EvalPrediction
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

In [4]:
def countNonZeroItems(items):
    nonZero = torch.nonzero(items, as_tuple= True)
    return len(nonZero[0])

Segons el notebook:
Fine-tuning BERT (and friends) for multi-label text classification.ipynb

https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb#scrollTo=HgpKXDfvKBxn  

In [5]:
# ensures reproducibility
torch.manual_seed(0)

In [6]:
# load labels
labelLoader = LabelLoader(baseDir)
len(labelLoader.labels)

7201

In [7]:
# Test dataset
baseDir = '/content/drive/My Drive/TFM-MUECIM'
ds = EURLEX57KDataset(baseDir,'FilesIndexShort.txt')
for i in range(5):
    item = ds.__getitem__(i)
    print(i, 'item["input_ids"]: ', item["input_ids"].shape)
    print(i, 'item["attention_mask"]: ', item["attention_mask"].shape)
    print(i, 'Nonzero masks: ', countNonZeroItems(item["attention_mask"]))
    print(i, 'item["labels"]: ', item["labels"])
    print(i, 'Nonzero labels: ', countNonZeroItems(item["labels"]))
    print('data (5 first elements): ', item['input_ids'][0:5],'\n')

Loading bert-base-cased tokenizer.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

0 item["input_ids"]:  torch.Size([512])
0 item["attention_mask"]:  torch.Size([512])
0 Nonzero masks:  67
0 item["labels"]:  tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)
0 Nonzero labels:  6
data (5 first elements):  tensor([ 5706,   120, 17881,   120, 16028]) 

1 item["input_ids"]:  torch.Size([512])
1 item["attention_mask"]:  torch.Size([512])
1 Nonzero masks:  53
1 item["labels"]:  tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)
1 Nonzero labels:  4
data (5 first elements):  tensor([ 2827, 22575,   113, 16028,   114]) 

2 item["input_ids"]:  torch.Size([512])
2 item["attention_mask"]:  torch.Size([512])
2 Nonzero masks:  29
2 item["labels"]:  tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)
2 Nonzero labels:  3
data (5 first elements):  tensor([ 2827, 22575,   113, 16028,   114]) 

3 item["input_ids"]:  torch.Size([512])
3 item["attention_mask"]:  torch.Size([512])
3 Nonzero masks:  33
3 item["labels"]:  tensor([0., 0., 0.,  ..., 0., 0., 0.], dt

In [8]:
ds = EURLEX57KDataset(baseDir,'FilesIndex.txt')
trainData, valData, testData = random_split(ds, [45000, 6000, 6000])

Loading bert-base-cased tokenizer.


In [9]:
# set batch size
batchSize = 10

# create dataloaders
trainDataLoader = DataLoader(trainData, batch_size=batchSize, shuffle=True)
valDataLoader = DataLoader(valData, batch_size=batchSize, shuffle=True)
testDataLoader = DataLoader(testData, batch_size=batchSize, shuffle=True)

In [11]:
# iterate through val batches
for i, batch in enumerate(valDataLoader):
  print(f'Batch {i}: ')
  batchFileNames = batch.get('fileName')
  batchData = batch.get('input_ids')
  batchAttentionMasks = batch.get('attention_mask')
  batchLabels = batch.get('labels')

  for elem in zip(batchFileNames, batchData, batchAttentionMasks, batchLabels):
    print(f'fileName: {elem[0]}')
    print(f'input_ids (5 first elements):\n{elem[1][0:5]}')
    print(f'attention_masks (5 first elements):\n{elem[2][0:5]}')
    print(f'Nonzero labels:{countNonZeroItems(elem[3])}\n')

  break

print('Done!')

Batch 0: 
fileName: data/datasets/EURLEX57K/test/32011D0396.json
input_ids (5 first elements):
tensor([1349,  120, 3614, 1545,  120])
attention_masks (5 first elements):
tensor([1, 1, 1, 1, 1])
Nonzero labels:5

fileName: data/datasets/EURLEX57K/train/31989R1384.json
input_ids (5 first elements):
tensor([ 2827, 22575,   113,   142,  8231])
attention_masks (5 first elements):
tensor([1, 1, 1, 1, 1])
Nonzero labels:3

fileName: data/datasets/EURLEX57K/train/32011R1340.json
input_ids (5 first elements):
tensor([22575,   113,  7270,   114,  1302])
attention_masks (5 first elements):
tensor([1, 1, 1, 1, 1])
Nonzero labels:5

fileName: data/datasets/EURLEX57K/train/32007R0060.json
input_ids (5 first elements):
tensor([ 2827, 22575,   113, 16028,   114])
attention_masks (5 first elements):
tensor([1, 1, 1, 1, 1])
Nonzero labels:3

fileName: data/datasets/EURLEX57K/train/32011L0015.json
input_ids (5 first elements):
tensor([ 2827, 15974,  2109,  1349,   120])
attention_masks (5 first elements)

In [12]:
len(labelLoader.labels)

7201

In [14]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained('google-bert/bert-base-cased')


config.num_labels = 7201


In [15]:
# bert huggingface pretrained model
from tfm_LabelIndex import LabelIndex

labelIndex = LabelIndex(baseDir)

model = AutoModelForSequenceClassification.from_pretrained(
    'google-bert/bert-base-cased',
    problem_type='multi_label_classification',
    num_labels=len(labelLoader.labels),
    id2label=labelIndex.id2label,
    label2id=labelIndex.label2id
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [17]:
# forward pass. no training. test case
item = trainData.__getitem__(0)

outputs = model(
    input_ids=item['input_ids'][0:512].unsqueeze(0),
    attention_mask=item['attention_mask'][0:512].unsqueeze(0),
    labels=item['labels'].unsqueeze(0))

In [18]:
outputs.logits[0]


tensor([-0.3062,  0.1514, -0.5521,  ...,  0.1682, -0.0794,  0.2346],
       grad_fn=<SelectBackward0>)

In [19]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
# calculate metrics

import numpy as np
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(outputs.logits[0])
threshold = 0.5
y_pred = np.zeros(probs.shape)
y_pred[np.where(probs >= threshold)] = 1
y_true = item['labels'].cpu().numpy() # Convert y_true to a NumPy array
f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
accuracy = accuracy_score(y_true, y_pred)

metrics = {'f1': f1_micro_average,
           'roc_auc': roc_auc,
           'accuracy': accuracy}


In [20]:
metrics

{'f1': 0.49576447715595057,
 'roc_auc': 0.6477765425236243,
 'accuracy': 0.49576447715595057}

In [ ]:
# training

import numpy as np
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score


def multi_label_metrics(predictions, labels, ):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_true = labels
    y_pred[np.where(probs >= 0.5)] = 1
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # define dictionary of metrics to return
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        references=p.label_ids)
    return result

# metric
metricName = 'f1'

# training arguments
trainArgs = TrainingArguments(
    'bert-base-cased',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batchSize,
    per_device_eval_batch_size=batchSize,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metricName)

trainer = Trainer(
    model=model,
    args=trainArgs,
    train_dataset=trainData,
    eval_dataset=valData,
    compute_metrics = compute_metrics,
    #data_collator = Data_Processing(),

)

# API-KEY-WAND-LIBRARY: 1bb618394e7e44feab7f79534fa2be428243d1bb

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Move the model to the correct device before training.
model.to(device)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: WARNING Serializing object of type dict that is 295000 bytes
wandb: WARNING Serializing object of type dict that is 295000 bytes


Epoch,Training Loss,Validation Loss
